In [36]:
from SMT_model import *
from time import time as time_clock
from z3 import IntNumRef
import json

In [37]:
#Demo with instance 01
Path = ('../Instances/inst07.dat')

timeout = 150


In [38]:
#Read the file from the path
with open(Path, 'r') as file:
    data = file.read()

instance = data_parsing(data)
print(instance)

{'m': 6, 'n': 17, 'l': [190, 185, 185, 190, 195, 185], 's': [11, 11, 23, 16, 2, 1, 24, 14, 20, 10, 11, 22, 2, 12, 9, 21, 10], 'D': [[0, 20, 19, 28, 58, 48, 45, 32, 90, 61, 71, 59, 65, 46, 72, 51, 46, 66], [103, 0, 81, 107, 38, 110, 55, 94, 76, 123, 88, 76, 69, 86, 99, 113, 108, 106], [73, 41, 0, 80, 40, 29, 26, 13, 75, 42, 70, 58, 46, 27, 53, 32, 27, 47], [28, 48, 47, 0, 55, 59, 52, 60, 62, 58, 43, 31, 40, 21, 59, 48, 53, 61], [83, 38, 79, 87, 0, 102, 58, 92, 56, 103, 68, 56, 49, 67, 79, 94, 106, 86], [76, 44, 72, 83, 43, 0, 98, 16, 46, 64, 73, 61, 73, 67, 27, 40, 30, 43], [65, 55, 26, 64, 58, 55, 0, 39, 69, 68, 50, 38, 26, 31, 66, 58, 53, 56], [60, 28, 56, 67, 27, 16, 82, 0, 62, 48, 75, 63, 69, 51, 40, 24, 14, 34], [58, 76, 77, 81, 55, 46, 69, 62, 0, 56, 43, 31, 43, 53, 23, 80, 76, 30], [74, 79, 42, 84, 71, 35, 68, 51, 56, 0, 59, 47, 54, 37, 59, 24, 65, 43], [15, 35, 34, 43, 12, 43, 41, 47, 39, 47, 0, 12, 24, 10, 16, 37, 61, 32], [27, 47, 46, 50, 24, 55, 38, 59, 31, 47, 12, 0, 12, 22,

In [39]:
def run_smt(instance, timeout, sb=False):
    generation_start_time = time_clock()

    # Build the SMT model
    o, x, max_distance = stm_model(instance, sb)
    generation_duration = time_clock() - generation_start_time
    o.set("timeout", int(timeout - generation_duration) * 1000)

    # Minimize the objective
    obj = o.minimize(max_distance)
    res = o.check()  # Check satisfiability
    final_time = int(time_clock() - generation_start_time)

    if res == sat:
        try:
            # Format the solution if satisfiable
            result_formatted = format_solution(instance, o.model(), x)
            return result_formatted, True, obj.value(), final_time
        except Exception as e:
            return str(e), False, obj.value()
    elif res == unknown:
        try:
            model = o.model()
            print(model)
            if model:  # Check if a model exists
                # Format the partial solution
                result_formatted = format_solution(instance, model, x)
                best_objective = model.eval(max_distance, model_completion=False)
                print(max_distance)
                return (
                    result_formatted,
                    False,
                    best_objective.as_long(),
                    final_time
                )
            else:
                # No model available, return fallback message
                return "No solution found", False, None
        except Exception as e:
            return f"unknown\nError retrieving model: {e}", False, None
    elif res == unsat:
        return "unsat", False, None
    else:
        return "unknown", False, None

In [ ]:
print(run_smt(instance, timeout, False))

In [40]:
def all_solutions(solvers, solutions):
    def convert(obj):
        if isinstance(obj, IntNumRef):
            return obj.as_long()
        if isinstance(obj, list):
            return [convert(i) for i in obj]
        return obj
    
    output_data = {}
    sol_index=0
    for solver in solvers:
        solution = solutions[sol_index]
        if solution == "unsat":
            print("Problem is unsatisfiable for solver", solver)
        elif solution == "unknown":
            print("Problem is unknown for solver", solver)
        elif solution[0] == "No solution found":
            print("No solution found for solver", solver)
        else:
            output_data.update({
                solver:
                    {
                    "time": convert(solution[3]),
                    "optimal": convert(solution[1]),
                    "obj": convert(solution[2]),
                    "sol": convert(solution[0])
                    }
                })
        sol_index += 1
        
    return output_data

In [41]:
def save_solution(solvers ,solutions, path):
    output_data = all_solutions(solvers, solutions)
    with open(path, 'w') as file:
        if output_data != []:
            print(output_data)
            json.dump(output_data, file, indent=4)

In [42]:
#Create a function that runs the SMT model for all the instances and saves the solutions in json files.
def run_all_instances():
    for i in range(1, 2):
        if i < 10:
            Path = f'../Instances/inst0{i}.dat'
        else:
            Path = f'../Instances/inst{i}.dat'
        if i < 30:
            with open(Path, 'r') as file:
                data = file.read()
            instance = data_parsing(data)
            solution_smt = run_smt(instance, timeout, False)
            solution_smt_sb = run_smt(instance, timeout, True)
            save_solution(["smt", "smt_sb"], [solution_smt, solution_smt_sb], f'res/SMT/inst{i}/inst{i}.json')

In [14]:
run_all_instances()

{'smt': {'time': 1, 'optimal': True, 'obj': 14, 'sol': [[4, 3, 1], [2, 5, 6]]}, 'smt_sb': {'time': 0, 'optimal': True, 'obj': 14, 'sol': [[4, 3, 1], [2, 5, 6]]}}


In [43]:
def run_single_instance(n):
        if n < 10:
            Path = f'../Instances/inst0{n}.dat'
        else:
            Path = f'../Instances/inst{n}.dat'
        with open(Path, 'r') as file:
            data = file.read()
        instance = data_parsing(data)
        solution_smt = run_smt(instance, timeout, False)
        #solution_smt_sb = run_smt(instance, timeout, True)
        solution_smt_sb = "unsat"
        if n < 10:
            save_solution(["smt", "smt_sb"], [solution_smt, solution_smt_sb], f'res/SMT/inst0{n}/inst0{n}.json')
        else:
            save_solution(["smt", "smt_sb"], [solution_smt, solution_smt_sb], f'res/SMT/inst{n}/inst{n}.json')

In [44]:
run_single_instance(7)

[x_0_5 = 17,
 x_4_14 = 17,
 x_5_4 = 17,
 x_0_16 = 12,
 x_4_7 = 17,
 x_4_6 = 17,
 x_4_9 = 3,
 x_1_5 = 17,
 x_3_15 = 17,
 x_5_12 = 17,
 x_4_11 = 17,
 x_5_11 = 17,
 x_5_9 = 17,
 x_1_6 = 17,
 x_4_4 = 17,
 x_1_7 = 17,
 x_5_8 = 17,
 x_2_7 = 10,
 x_5_10 = 17,
 x_2_16 = 17,
 x_3_4 = 17,
 x_4_5 = 17,
 x_5_13 = 17,
 x_0_4 = 17,
 x_4_12 = 17,
 x_1_2 = 17,
 x_2_9 = 16,
 x_4_1 = 17,
 x_2_2 = 17,
 x_2_13 = 17,
 x_5_5 = 1,
 x_4_16 = 17,
 x_1_8 = 17,
 x_1_4 = 17,
 x_1_10 = 17,
 x_0_13 = 17,
 x_1_14 = 17,
 x_2_10 = 17,
 x_0_1 = 17,
 x_2_5 = 11,
 x_5_14 = 17,
 x_3_3 = 17,
 x_1_11 = 17,
 x_0_11 = 17,
 x_5_3 = 17,
 x_1_12 = 17,
 x_3_8 = 17,
 x_2_3 = 17,
 x_1_9 = 17,
 x_3_16 = 17,
 x_1_16 = 17,
 x_3_5 = 17,
 s = Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(K(Int,
                                        11),
                                        7,
                                        14),
                                        6,
                          